In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os

import pandas as pd
import numpy as np
import math

In [2]:
expeditionsSheet_ID = '1toCpXPNdHbC1qH6K88a6xDdoVQr2X0aHA4tG5DyQuj0'
expeditionsDataRange = 'Sheet1'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [3]:
creds = None
expCredsPath = 'expeditions.pkl'
if os.path.exists(expCredsPath):
    with open(expCredsPath, 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open(expCredsPath, 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId = expeditionsSheet_ID,
                           range=expeditionsDataRange).execute()
values = result.get('values', [])

In [4]:
def standardHTML(name, desc, img, expedition, link):
    title = name
    altTitle = name
    imageURL = img
    description = desc
    html = ""
    if link == "":
        link = None
    
    if expedition:
        title += "*"
    html += "<!-- wp:html -->\n"
    html += "<div class=\"personnel\">\n"
    html += '<p><img class="profileimg" title="%s" src="%s" alt="%s"></p>\n' % (altTitle, imageURL, altTitle)
    if link:
        html += '<p class="projecttitle"><a href="%s">%s</a></p>\n' % (link, title)
    else:
        html += '<p class="projecttitle">%s</p>\n' % (title)
    html += '<p class="projectdescription">%s</p>\n' % (description)
    html += '</div>\n'
    html += '<!-- /wp:html -->\n'
    html += '\n'
    return html

def minimalHTML(name, desc, img, expedition, link):
    title = name
    html = ""
    if expedition:
        title += "*"
    
    html += '<li>%s - %s</li>\n' % (title, desc)
    return html

def noOutput(name, desc, img, expedition, link):
    return ""

In [5]:
baseLevels = {
    1: "Project Candidate",
    2: "Technical Contributor",
    3: "Research Contributor",
    4: "Project Lead",
    5: "Staff Engineer",
    6: "E4E Director",
    22: "Technical Contributor - Alumni",
    23: "Research Contributor - Alumni",
    24: "Project Lead - Alumni",
    25: "Former Staff Engineer",
    26: "Former E4E Director"
}

expeditionLevels = {
    3: "Expedition Member",
    4: "Expedition Student Leader",
    5: "Staff Engineer",
    6: "E4E Director",
    23: "Expedition Alumni",
    24: "Expedition Leader Alumni",
    25: "Former Staff Engineer",
    26: "Former E4E Director"
}

baseLevelHeadings = {
    1: "Project Candidates",
    2: "Technical Contributors",
    3: "Research Contributors",
    4: "Project Leads",
    5: "Staff Engineers",
    6: "E4E Directors",
    22: "Technical Contributor - Alumni",
    23: "Research Contributor - Alumni",
    24: "Project Lead - Alumni",
    25: "Former Staff Engineers",
    26: "Former E4E Directors"
}

levelFormatter = {
    1: noOutput,
    2: minimalHTML,
    3: standardHTML,
    4: standardHTML,
    5: standardHTML,
    6: standardHTML,
    22: minimalHTML,
    23: standardHTML,
    24: standardHTML,
    25: standardHTML,
    26: standardHTML
}

sectionOutput = {
    1: ("", ""),
    2: ("<!-- wp:list -->\n<ul>\n", "</ul>\n<!-- /wp:list -->\n"),
    3: ("", ""),
    4: ("", ""),
    5: ("", ""),
    6: ("", ""),
    22: ("<!-- wp:list -->\n<ul>\n", "</ul>\n<!-- /wp:list -->\n"),
    23: ("", ""),
    24: ("", ""),
    25: ("", ""),
    26: ("", "")
}

peoplePageOrder = [
    6, 5, 4, 3, 2
]

alumniPageOrder = [
    26, 25, 24, 23, 22
]

blankProfilePic = "http://e4e.ucsd.edu/wp-content/uploads/blank-profile-drawing.png"

In [6]:
df = pd.DataFrame(values[1:], columns=values[0])

In [7]:
df[["Level", "End year", "Start year"]] = df[["Level", "End year", "Start year"]].apply(pd.to_numeric)
df['Expeditionary'] = df['Expeditionary'].map({"TRUE": True, "FALSE": False})
df['Image'] = df['Image'].replace(['', None], [blankProfilePic, blankProfilePic])
df['End year'].fillna(9999, inplace=True)
df["Link"].fillna(value="", inplace=True)
df["Description"].fillna(value="", inplace=True)

df.sort_values(['Level', 'End year', 'Start year'], inplace=True, ascending=[True, False, False])

In [8]:
html = ''
html += "<!-- wp:paragraph -->\n"
html += "<p>* Denotes people that have participated in an expedition.</p>\n"
html += "<!-- /wp:paragraph -->\n"
html += "\n"
for level in peoplePageOrder:
    people = df[df['Level'] == level]
    if len(people) > 0:
        html += "<!-- wp:heading -->\n"
        html += "<h2>%s</h2>\n" % (baseLevelHeadings[level])
        html += "<!-- /wp:heading -->\n"
        html += "\n"

        html += sectionOutput[level][0]
    
    for personIdx in people.index:
        title = people.loc[personIdx, "Title"]
        altTitle = people.loc[personIdx, "Title"]
        imageURL = people.loc[personIdx, "Image"]
        description = people.loc[personIdx, "Description"]
        expedition = people.loc[personIdx, "Expeditionary"]
        link = people.loc[personIdx, "Link"]
        html += levelFormatter[level](title, description, imageURL, expedition, link)
        
    html += sectionOutput[level][1]
print(html)
with open("people.txt", 'w') as file:
    file.write(html)

<!-- wp:paragraph -->
<p>* Denotes people that have participated in an expedition.</p>
<!-- /wp:paragraph -->

<!-- wp:heading -->
<h2>E4E Directors</h2>
<!-- /wp:heading -->

<!-- wp:html -->
<div class="personnel">
<p><img class="profileimg" title="Dr. Ryan Kastner" src="http://e4e.ucsd.edu/wp-content/uploads/ryan.jpg" alt="Dr. Ryan Kastner"></p>
<p class="projecttitle"><a href="http://kastner.ucsd.edu/">Dr. Ryan Kastner*</a></p>
<p class="projectdescription">E4E Founding Director since 2010</p>
</div>
<!-- /wp:html -->

<!-- wp:html -->
<div class="personnel">
<p><img class="profileimg" title="Dr. Curt Schurgers" src="http://e4e.ucsd.edu/wp-content/uploads/curt_150x150.png" alt="Dr. Curt Schurgers"></p>
<p class="projecttitle">Dr. Curt Schurgers*</p>
<p class="projectdescription">E4E Founding Director since 2010</p>
</div>
<!-- /wp:html -->

<!-- wp:html -->
<div class="personnel">
<p><img class="profileimg" title="Dr. Albert Lin" src="http://e4e.ucsd.edu/wp-content/uploads/albert.p

In [9]:
html = ''
html += "<!-- wp:paragraph -->\n"
html += "<p>* Denotes people that have participated in an expedition.</p>\n"
html += "<!-- /wp:paragraph -->\n"
html += "\n"
for level in alumniPageOrder:
    people = df[df['Level'] == level]
    if len(people) > 0:
        html += "<!-- wp:heading -->\n"
        html += "<h2>%s</h2>\n" % (baseLevelHeadings[level])
        html += "<!-- /wp:heading -->\n"
        html += "\n"

        html += sectionOutput[level][0]
    
    for personIdx in people.index:
        title = people.loc[personIdx, "Title"]
        altTitle = people.loc[personIdx, "Title"]
        imageURL = people.loc[personIdx, "Image"]
        description = people.loc[personIdx, "Description"]
        expedition = people.loc[personIdx, "Expeditionary"]
        link = people.loc[personIdx, "Link"]
        html += levelFormatter[level](title, description, imageURL, expedition, link)
        
    html += sectionOutput[level][1]
print(html)
with open("alumni.txt", 'w') as file:
    file.write(html)

<!-- wp:paragraph -->
<p>* Denotes people that have participated in an expedition.</p>
<!-- /wp:paragraph -->

<!-- wp:heading -->
<h2>Former Staff Engineers</h2>
<!-- /wp:heading -->

<!-- wp:html -->
<div class="personnel">
<p><img class="profileimg" title="Eric Lo" src="http://e4e.ucsd.edu/wp-content/uploads/2016_eric.jpg" alt="Eric Lo"></p>
<p class="projecttitle">Eric Lo*</p>
<p class="projectdescription">E4E Staff Engineer, 2014-2019</p>
</div>
<!-- /wp:html -->

<!-- wp:heading -->
<h2>Project Lead - Alumni</h2>
<!-- /wp:heading -->

<!-- wp:html -->
<div class="personnel">
<p><img class="profileimg" title="Dominique Meyer" src="http://e4e.ucsd.edu/wp-content/uploads/blank-profile-drawing.png" alt="Dominique Meyer"></p>
<p class="projecttitle">Dominique Meyer*</p>
<p class="projectdescription"></p>
</div>
<!-- /wp:html -->

<!-- wp:html -->
<div class="personnel">
<p><img class="profileimg" title="Nicholas Munoz" src="http://e4e.ucsd.edu/wp-content/uploads/2020_nmunoz.jpg" alt="